# Throwing Darts

<img src="media/05_Darts_Presentation.png" width="500">  

<img src="media/06_Darts.png" width="300">  

<img src="media/07_Darts.png" width="300">  

# Categorical Fields

In [52]:
import pandas as pd

satanas = pd.DataFrame({'a': [1, 2, 3, 1, 12], 'b': [4, 5, 6, 2, 3], 'target': ['one', 'two', 'three', 'one', 'two']})

In [53]:
satanas.target = pd.Categorical(satanas.target)
satanas['target'] = satanas.target.cat.codes
satanas

,a,b,target
0,1,4,0
1,2,5,2
2,3,6,1
3,1,2,0
4,12,3,2


In [54]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(satanas.target)
y

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

# A multi-class model

In [55]:

# You're going to build a model that predicts who threw which dart only based on where that dart landed! (That is the dart's x and y coordinates on the board.)

# This problem is a multi-class classification problem since each dart can only be thrown by one of 4 competitors. So classes/labels are mutually exclusive, and therefore we can build a neuron with as many output as competitors and use the softmax activation function to achieve a total sum of probabilities of 1 over all competitors.

# The Sequential model and Dense layers are already imported for you to use.

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [57]:
# Instantiate a sequential model
model = Sequential()
  
# Add 3 dense layers of 128, 64 and 32 neurons each
model.add(Dense(128, input_shape=(2,), activation='relu'))
model.add(Dense( 64, activation='relu'))
model.add(Dense( 32, activation='relu'))
  
# Add a dense layer with as many neurons as competitors
model.add(Dense(4, activation="softmax"))
  
# Compile your model using categorical_crossentropy loss
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

In [58]:
darts = pd.read_csv('datasets/darts.csv')
darts.head()

,xCoord,yCoord,competitor
0,0.196451,-0.520341,Steve
1,0.476027,-0.306763,Susan
2,0.003175,-0.980736,Michael
3,0.294078,0.267566,Kate
4,-0.051120,0.598946,Steve


In [59]:
# Transform into a categorical variable
darts.competitor = pd.Categorical(darts.competitor)

# Assign a number to each category (label encoding)
darts.competitor = darts.competitor.cat.codes 

# Import to_categorical from keras utils module
from tensorflow.keras.utils import to_categorical

coordinates = darts.drop(['competitor'], axis=1)
# Use to_categorical on your labels
competitors = to_categorical(darts.competitor)

# Now print the one-hot encoded labels
print('One-hot encoded competitors: \n',competitors)

One-hot encoded competitors: 
 [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [60]:
from sklearn.model_selection import train_test_split
coord_train, competitors_train, coord_test, competitors_test = train_test_split(
    coordinates, competitors, test_size=0.2, random_state=42
)

In [61]:
# Fit your model to the training data for 200 epochs
model.fit(coord_train, competitors_train,epochs=200)

# Evaluate your model accuracy on the test data
accuracy = model.evaluate(coord_test, competitors_test)[1]

# Print accuracy
print('Accuracy:', accuracy)

ValueError: Data cardinality is ambiguous:
  x sizes: 640
  y sizes: 160
Make sure all arrays contain the same number of samples.

In [ ]:
# Predict on coords_small_test
preds = model.predict(coords_small_test) # Não tem

# Print preds vs true values
print("{:45} | {}".format('Raw Model Predictions','True labels'))
for i,pred in enumerate(preds):
  print("{} | {}".format(pred,competitors_small_test[i]))

# Extract the position of highest probability from each pred vector
preds_chosen = [np.argmax(pred) for pred in preds]

# Print preds vs true values
print("{:10} | {}".format('Rounded Model Predictions','True labels'))
for i,pred in enumerate(preds_chosen):
  print("{:25} | {}".format(pred,competitors_small_test[i]))